In [1]:
#Import Libraries
import os
import json
import csv
import matplotlib.pyplot as plt
import sklearn
import tensorflow
import numpy as np
import pandas as pd
import time
import datetime
import gc
import random
import re
import torch
import torch.nn as nn
import time
import spacy
import kagglehub

#NLP Packages
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Dense
from tensorflow.keras.layers import Flatten, LSTM
from tensorflow.keras.layers import GlobalMaxPooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Concatenate
from nltk.corpus import stopwords
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler,random_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import MultiLabelBinarizer
import transformers
from transformers import BertForSequenceClassification, AdamW, BertConfig,BertTokenizer,get_linear_schedule_with_warmup
from IPython.display import display, clear_output

import pyarrow


2025-02-27 23:46:54.366485: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-27 23:46:54.390857: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740718014.412809  508075 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740718014.418092  508075 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-27 23:46:54.442881: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

# Model

In [ ]:
class Model:
    def __init__(self, num_labels, train_dataloader):
        self.num_labels = num_labels
        self.train_dataloader = train_dataloader
        # Load BertForSequenceClassification, the pretrained BERT model with a single 
        # linear classification layer on top. 
        self.model = BertForSequenceClassification.from_pretrained(
            "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
            num_labels = num_labels, # The number of output labels--2 for binary classification.
                            # You can increase this for multi-class tasks.   
            output_attentions = False, # Whether the model returns attentions weights.
            output_hidden_states = False, # Whether the model returns all hidden-states.
            problem_type = "multi_label_classification" # Defaults loss function to BCEWithLogitsLoss
        )

        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        print("Device: ", device)

        # if device == "cuda:0":
        # # Tell pytorch to run this model on the GPU.
        #     model = model.cuda()
        self.model = self.model.to(device)

        self.optimizer = torch.optim.AdamW(self.model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

        # Define Loss function Compatible with Multi-label classification may be redundant givent that "problem_type" specified as multi-label-classification in model
        self.criterion = nn.BCEWithLogitsLoss()

        # Number of training epochs. The BERT authors recommend between 2 and 4. 
        # We chose to run for 4, but we'll see later that this may be over-fitting the
        # training data.
        epochs = 1

        # Total number of training steps is [number of batches] x [number of epochs]. 
        # (Note that this is not the same as the number of training samples).
        total_steps = len(train_dataloader) * epochs

        # Create the learning rate scheduler.
        self.scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                    num_warmup_steps = 0, # Default value in run_glue.py
                                                    num_training_steps = total_steps)
    
    ###########################
# Train with training set #
###########################
def train(self, iterator, optimizer, criterion, device, scheduler, epoch):
    
    # Enter Train Mode
    self.model.train()
    train_loss = 0

    #Number of iterations equal to total train dataset / batch size
    for step, batch in enumerate(iterator):
        #Print progress in epoch
        print(f"Progress: {step+1}/{len(iterator)}", end='\r')
        # Parse iterator tensor dataset for important information
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        output = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)
        
        # Generate prediction
        optimizer.zero_grad()
        
        # Compute gradients and update weights
        loss = criterion(output.logits, b_labels) # BCEWithLogitsLoss has sigmoid
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        # accumulate train loss
        train_loss += loss
    
    # print completed result
    print()
    print('Train Loss: %f' % (train_loss))
    return train_loss

#############################
# Validate with testing set #
#############################
def test(model, iterator, optimizer, criterion, device, epoch):

    # Enter Evaluation Mode
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for step, batch in enumerate(iterator):
        
            #Print progress in epoch
            print(f"Progress: {step+1}/{len(iterator)}", end='\r')
            print(f'Time per iteration: {datetime.timedelta(seconds=(time.time() - start_time)//(step+1))}', end='\r')
            
            # Parse iterator tensor dataset for important information
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)
            
            # generate prediction
            output = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)
            prob = output.logits.sigmoid()   # BCEWithLogitsLoss has sigmoid
            
            # record processed data count
            total += (b_labels.size(0)*b_labels.size(1))

            # take the index of the highest prob as prediction output
            THRESHOLD = 0.7
            prediction = prob.detach().clone()
            prediction[prediction > THRESHOLD] = 1
            prediction[prediction <= THRESHOLD] = 0
            correct += prediction.eq(b_labels).sum().item()

        print()
    
    #print completed result
    acc = 100.*correct/total
    print('Correct: %i  / Total: %i / Test Accuracy: %f' % (correct, total, acc))
    return acc